# Chat bot 1: QA system using similarity scores


In [45]:
#import packages
import pandas as pd
import numpy as np

#import nltk
import nltk

#import other
import string
import random

In [46]:
#importing the data

data=open('data.txt','r',errors = 'ignore')
text=data.read()

In [47]:
print(text)

Data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from structured and unstructured data,[1][2] and apply knowledge and actionable insights from data across a broad range of application domains. Data science is related to data mining, machine learning and big data.

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" in order to "understand and analyze actual phenomena" with data.[3] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge. However, data science is different from computer science and information science. Turing Award winner Jim Gray imagined data science as a "fourth paradigm" of science (empirical, theoretical, computational, and now data-driven) and asserted that "everything about science is changing because of the impact of 

In [48]:
# to lower  case
text=text.lower()

# tokenize the words
wr_ids = nltk.word_tokenize(text)

# tokenize the sent 
st_ids = nltk.sent_tokenize(text)

In [49]:
wr_ids[0]

'data'

In [50]:
st_ids[0]

'data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from structured and unstructured data,[1][2] and apply knowledge and actionable insights from data across a broad range of application domains.'

In [51]:
#importing the libraries

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
#define function for response

def get_output(user_input):
    
    #define the output
    output=''
    
    #append input to text
    st_ids.append(user_input)
    
    #define tfidf
    txt_v = TfidfVectorizer(stop_words='english')
    
    #get vector
    vec_txt = txt_v.fit_transform(st_ids)
    
    #get score
    rank_score= cosine_similarity(vec_txt[-1], vec_txt)
    
    
    idx=rank_score.argsort()[0][-2]
    ft_out = rank_score.flatten()
    ft_out.sort()
    final_v = ft_out[-2]
    
    
    if(final_v==0):
        output=output+"Dont know this annswer, Ask something else"
        return output
    else:
        output = output+st_ids[idx]
        return output


In [53]:
# Final code to run the bot

print("Enter your question")
print("")
in_txt = input()
in_txt=in_txt.lower()
print("")
print("Ans:", get_output(in_txt))
print("")
st_ids.remove(in_txt)

Enter your question

What is Data Science?

Ans: however, data science is different from computer science and information science.



# # Chat bot 2 : Context based Chatbot 

In [ ]:
!pip install transformers

In [ ]:
#import model and tokenizer
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

# import torch 
import torch

#loading the pre trained Model
qna_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#loading the Tokenizer for same model
qna_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
ans_text = """Virat Kohli (Hindustani: [ʋɪˈraːʈ ˈkoːɦliː] (About this soundlisten); born 5 November 1988) is an Indian cricketer and the current captain of the India national team. A right-handed top-order batsman. He is widely regarded as one of the greatest batsman of present era. He plays for Delhi in domestic cricket and for Royal Challengers Bangalore in the Indian Premier League (IPL) as captain of the franchise since 2013. He was a part of Indian cricket team which won the 2011 crickket world cup

Kohli captained India Under-19s to victory at the 2008 Under-19 World Cup in Malaysia. After a few months later, he made his ODI debut for India against Sri Lanka at the age of 19. Initially having played as a reserve batsman in the Indian team, he soon established himself as a regular in the ODI middle-order and was part of the squad that won the 2011 World Cup. He made his Test debut in 2011 and shrugged off the tag of "ODI specialist" by 2013 with Test hundreds in Australia and South Africa.[3] Having reached the number one spot in the ICC rankings for ODI batsmen for the first time in 2013,[4] Kohli also found success in the Twenty20 format, winning the Man of the Tournament twice at the ICC World Twenty20 (in 2014 and 2016)."""


In [ ]:
#function to get answer for a use given question

def QnA(user_input_que):
  
  #tokinizng the texts
  in_tok = qna_tokenizer.encode_plus(user_input_que, ans_text, return_tensors="pt") 

  #getting scores from tokens
  ans_str_sc, ans_en_sc = qna_model(**in_tok,return_dict=False)

  #getting the position
  ans_st = torch.argmax(ans_str_sc)  
  ans_en = torch.argmax(ans_en_sc) + 1  

  #ids are then converted to tokens
  ans_tok = qna_tokenizer.convert_ids_to_tokens(in_tok["input_ids"][0][ans_st:ans_en])

  #getting the answer
  return qna_tokenizer.convert_tokens_to_string(ans_tok)

In [ ]:
que = "when did kohli win world cup"

QnA(que)

In [ ]:
que = "when did kohli born"

QnA(que)

In [ ]:
ans_text = """Coronavirus disease 2019 (COVID-19) is a contagious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first known case was identified in Wuhan, China, in December 2019.[7] The disease has since spread worldwide, leading to an ongoing pandemic.[8]

Symptoms of COVID-19 are variable, but often include fever,[9] cough, headache,[10] fatigue, breathing difficulties, and loss of smell and taste.[11][12][13] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[14] Of those people who develop symptoms noticeable enough to be classed as patients, most (81%) develop mild to moderate symptoms (up to mild pneumonia), while 14% develop severe symptoms (dyspnea, hypoxia, or more than 50% lung involvement on imaging), and 5% suffer critical symptoms (respiratory failure, shock, or multiorgan dysfunction).[15] Older people are at a higher risk of developing severe symptoms. Some people continue to experience a range of effects (long COVID) for months after recovery, and damage to organs has been observed.[16] Multi-year studies are underway to further investigate the long-term effects of the disease.[16]

COVID-19 transmits when people breathe in air contaminated by droplets and small airborne particles containing the virus. The risk of breathing these in is highest when people are in close proximity, but they can be inhaled over longer distances, particularly indoors. Transmission can also occur if splashed or sprayed with contaminated fluids in the eyes, nose or mouth, and, rarely, via contaminated surfaces. People remain contagious for up to 20 days, and can spread the virus even if they do not develop symptoms.[17][18]"""


In [ ]:
que = "what are the symptoms of covid19?"

QnA(que)